<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/viewers_deployment/Creating_Google_Healthcare_DICOM_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites

This notebook complements the tutorial available in this document: https://tinyurl.com/idc-ohif-gcp

You must complete the prerequisites discussed in that document before proceeding with this notebook!

Please use the document above for providing feedback or asking questions!

---
Updated: Feb 2026

## Preparation

You will need to authenticate with Google to be able to complete this notebook.

In [ ]:
from google.colab import auth
auth.authenticate_user()

Substitute `REPLACE_WITH_YOUR_PROJECT_ID` with the ID of the project you can access, which has billing configured, in the following cell and anywhere else you may see it.

In [ ]:
import os
os.environ["MY_PROJECT_ID"] = "REPLACE_WITH_YOUR_PROJECT_ID"

In [ ]:
!gcloud config set project ${MY_PROJECT_ID}

## Define the list of instances

We will use the query below to get the URLs for the files corresponding to the DICOM instances included in the sample DICOM study.

Same as in the previous step, make sure to replace `REPLACE_WITH_YOUR_PROJECT_ID` in the cell below with the ID of your project, which should have billing enabled.

In [ ]:
%%bigquery --project=REPLACE_WITH_YOUR_PROJECT_ID cohort_df

SELECT gcs_url
FROM `bigquery-public-data.idc_current.dicom_all`
WHERE StudyInstanceUID = "1.3.6.1.4.1.14519.5.2.1.6279.6001.224985459390356936417021464571"

## Retrieve DICOM instances

First, we will save the Google Cloud Storage (GCS) URLs into a separate file.

In [ ]:
cohort_df = cohort_df.join(cohort_df["gcs_url"].str.split('#', 1, expand=True).rename(columns={0:'gcs_url_no_revision', 1:'gcs_revision'}))
cohort_df["gcs_url_no_revision"].to_csv("gcs_paths.txt", header=False, index=False)

In [ ]:
!head /content/gcs_paths.txt

Next, we will download the files to the VM filesystem using the standard `gsutil` command, which is preinstalled on Colab instances.

IDC-hosted data is available from free Google Storage buckets maintained under [Google Public Dataset Program](https://console.cloud.google.com/marketplace/product/gcp-public-data-idc/nci-idc-data), which sponsors free egress of the data either within or out of the Google Cloud.

In [ ]:
# https://cloud.google.com/storage/docs/gsutil/commands/cp
%%capture
!mkdir downloaded_cohort
!cat gcs_paths.txt | gsutil -m cp -I ./downloaded_cohort

## Create and populate GCP Healthcare DICOM store

Next we will create a temporary storage bucket that will contain the files that will be imported into a DICOM store. Note that we could import the files directly, one by one, from their original locations, but that operation is slower.

In [ ]:
os.environ["MY_TEMP_BUCKET"] = "gs://"+os.environ["MY_PROJECT_ID"]+"_af_ohiftutorial_temp"
print("DICOM data bucket location: "+os.environ["MY_TEMP_BUCKET"])

**WARNING**: storage bucket names must be globally unique! If you fail to create a bucket due to a conflict, choose a different name, or reuse an existing bucket!

In [ ]:
!gsutil mb -p $MY_PROJECT_ID ${MY_TEMP_BUCKET}

In [ ]:
%%capture
!gsutil -m cp -r ./downloaded_cohort/* ${MY_TEMP_BUCKET}

In the following cells we are first creating a Healthcare dataset, and then a DICOM store within that dataset.

**WARNING**:
* please make sure Google Cloud Healthcare Service Agent has the Storage Viewer role to access the storage bucket! If you are not sure how to assign that role, please see the main tutorial document [here](https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#heading=h.jg76iucxtly9).
* if the dataset or DICOM store with the given names already exist, you can skip the following steps, or choose different names for those resources.

In [ ]:
# These uniquely identify your DICOM store:
# you will need those if you want to open content of this store in OHIF Viewer
os.environ["DATASET_ID"] = "af_ohiftutorial_temp_dataset"
os.environ["STORE_ID"] = "af_ohiftutorial_temp_store"
os.environ["LOCATION"] = "us-central1"

In [ ]:
!gcloud healthcare datasets create ${DATASET_ID} --project=${MY_PROJECT_ID} --location=${LOCATION}

In [ ]:
!gcloud healthcare dicom-stores create ${STORE_ID} --project=${MY_PROJECT_ID} --dataset=${DATASET_ID} --location=${LOCATION}

Finally, now that we have a GCS storage buckets with the files we want to use to populate the DICOM store, we can import the content of the bucket into the DICOM store created in the previous step.

In [ ]:
!gcloud healthcare dicom-stores import gcs ${STORE_ID} \
  --dataset=${DATASET_ID} \
  --location=${LOCATION} \
  --gcs-uri=${MY_TEMP_BUCKET}/**.dcm \
  --project=${MY_PROJECT_ID}

## Viewing DICOM store content in OHIF Viewer

If you completed deployment of the OHIF Viewer, as described in [the tutorial document](https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#heading=h.jg76iucxtly9), you should now be able to open the study we retrieved in the first steps of this notebook in the viewer.

To do this, update the string below to form the URL:

* MY_OHIF_VIEWER: Firebase hosting URL
* MY_PROJECT_ID: project under which you created the Healthcare dataset
* DATASET_ID: dataset ID you specified earlier while creating the dataset
* STORE_ID: DICOM store ID you specified earlier
* 1.3.6.1.4.1.14519.5.2.1.6279.6001.224985459390356936417021464571 is the DICOM StudyInstanceUID that we used in the beginning of this notebook to retrieve the study from IDC - you can replace this with the StudyInstanceUID corresponding to any study you have in the DICOM store.

**MY_OHIF_VIEWER_URL/projects/MY_PROJECT_ID/locations/LOCATION/datasets/DATASET_ID/dicomStores/STORE_ID/study/1.3.6.1.4.1.14519.5.2.1.6279.6001.224985459390356936417021464571**

Now is the good time to return back to the [tutorial document](https://docs.google.com/document/d/1v4Syu_yOV6yH--QBLGzsL9fJ7-XyD1CnQu4iTIoPVD8/edit#heading=h.jg76iucxtly9)!

## Support

If you have any questions about this notebook, please post your question on the [IDC User Forum](https://discourse.canceridc.dev) or [open an issue](https://github.com/ImagingDataCommons/IDC-Tutorials/issues/new) in the [IDC Tutorials repository](https://github.com/ImagingDataCommons/IDC-Tutorials).

## Acknowledgments

Imaging Data Commons has been funded in whole or in part with Federal funds from the National Cancer Institute, National Institutes of Health, under Task Order No. HHSN26110071 under Contract No. HHSN261201500003I.

If you use IDC in your research, please cite the following publication:

> Fedorov, A., Longabaugh, W. J. R., Pot, D., Clunie, D. A., Pieper, S. D., Gibbs, D. L., Bridge, C., Herrmann, M. D., Homeyer, A., Lewis, R., Aerts, H. J. W., Krishnaswamy, D., Thiriveedhi, V. K., Ciausu, C., Schacherer, D. P., Bontempi, D., Pihl, T., Wagner, U., Farahani, K., Kim, E. & Kikinis, R. _National Cancer Institute Imaging Data Commons: Toward Transparency, Reproducibility, and Scalability in Imaging Artificial Intelligence_. RadioGraphics (2023). [https://doi.org/10.1148/rg.230180](https://doi.org/10.1148/rg.230180)